# MySQL Database Migration (Old to New AWS RDS)

## Step 0 — 导入库

In [18]:
import os
import subprocess

## Step 1 — 设置路径与备份文件位置

In [19]:
# 当前工作目录
cwd = os.getcwd()
print(f"📂 当前工作目录: {cwd}")

# 备份文件路径：保存在当前目录下的 02_wrangled_data 文件夹
backup_dir = os.path.join(cwd, "02_wrangled_data")
os.makedirs(backup_dir, exist_ok=True)
backup_file = os.path.join(backup_dir, "plantx_backup.sql")

print(f"💾 备份文件将保存到: {backup_file}")

📂 当前工作目录: e:\05_YZH_DS\02_Monash_DS\2025_S2_FIT5120_Industry_Experience_Studio_Project\06_main_project\03_github_submission\03_github_submission\2025-08-SDG13-Plant-X-Website\01_data_wrangling
💾 备份文件将保存到: e:\05_YZH_DS\02_Monash_DS\2025_S2_FIT5120_Industry_Experience_Studio_Project\06_main_project\03_github_submission\03_github_submission\2025-08-SDG13-Plant-X-Website\01_data_wrangling\02_wrangled_data\plantx_backup.sql


## Step 2 — 数据库配置

In [20]:
# 旧数据库（原 AWS 账号）
old_db = {
    "host": "database-plantx.cqz06uycysiz.us-east-1.rds.amazonaws.com",
    "user": "zihan",
    "password": "2002317Yzh12138.",
    "database": "FIT5120_PlantX_Database"
}

# 新数据库（同学提供的新账号）
new_db = {
    "host": "database-plantx.cerawa8mwzg4.us-east-1.rds.amazonaws.com",
    "user": "zihan",
    "password": "2002317Yzh12138.",
    "database": "FIT5120_PlantX_Database"
}

## Step 3 — 指定 mysqldump/mysql 路径并检查

In [21]:
# 指定路径（你的 MySQL Workbench 在 E 盘）
MYSQL_BIN_PATH = r"E:\02_App\MySQL\MySQL Workbench 8.0 CE"
dump_exe = os.path.join(MYSQL_BIN_PATH, "mysqldump.exe")
mysql_exe = os.path.join(MYSQL_BIN_PATH, "mysql.exe")

# 检查路径是否存在
for exe in [dump_exe, mysql_exe]:
    if not os.path.exists(exe):
        print(f"⚠️ 警告：找不到 {exe}")
    else:
        print(f"✅ 找到: {exe}")

✅ 找到: E:\02_App\MySQL\MySQL Workbench 8.0 CE\mysqldump.exe
✅ 找到: E:\02_App\MySQL\MySQL Workbench 8.0 CE\mysql.exe


## Step 4 — 导出旧数据库

In [22]:
## Step 4 — 导出旧数据库
dump_cmd = [
    dump_exe,
    "-h", old_db["host"],
    "-u", old_db["user"],
    f"-p{old_db['password']}",
    "--routines", "--triggers", "--events", "--add-drop-table",
    old_db["database"]
]

print("\n🟡 正在导出旧数据库...")
#
# 【修正 1】: 使用 "wb" (二进制写入) 模式, 而不是 "w" 和 encoding="utf-8"
# 这将确保 mysqldump 输出的二进制数据 (如 GEOMETRY) 被正确保存。
#
with open(backup_file, "wb") as f:
    subprocess.run(dump_cmd, stdout=f, check=True)
print(f"✅ 导出完成: {backup_file}")


🟡 正在导出旧数据库...
✅ 导出完成: e:\05_YZH_DS\02_Monash_DS\2025_S2_FIT5120_Industry_Experience_Studio_Project\06_main_project\03_github_submission\03_github_submission\2025-08-SDG13-Plant-X-Website\01_data_wrangling\02_wrangled_data\plantx_backup.sql


## Step 5 — 导入到新数据库

In [23]:
## Step 5 — 导入到新数据库
print("\n🟡 正在导入到新数据库（改进版：忽略权限错误 + 关闭外键检查）...")

#
# 【修正 2】: 定义 import_cmd 变量
# 你在 subprocess.run 中使用了 import_cmd, 但它没有被定义。
# 我们还添加了 --default-character-set=utf8mb4 来确保连接编码正确。
#
import_cmd = [
    mysql_exe,
    "-h", new_db["host"],
    "-u", new_db["user"],
    f"-p{new_db['password']}",
    "--default-character-set=utf8mb4",
    new_db["database"]
]

#
# 【修正 3】: 移除 SET sql_log_bin=0;
# AWS RDS 不允许此操作，它会导致 ERROR 1227 权限错误。
#
prefix_sql = "SET FOREIGN_KEY_CHECKS=0;\n"

# 以二进制模式读取，这是正确的
with open(backup_file, "rb") as f:
    backup_data = f.read()

# 将 SQL 文件内容与前缀命令拼接
import_data = prefix_sql.encode("utf-8") + backup_data + b"\nSET FOREIGN_KEY_CHECKS=1;\n"

# 执行导入命令
result = subprocess.run(
    import_cmd + ["--force"],
    input=import_data,
    capture_output=True
)

# 输出错误（仅显示前 800 个字节以防太长）
print("STDERR（错误信息片段）:\n", result.stderr.decode(errors="ignore")[:800])

# 检查返回值
if result.returncode == 0:
    print("✅ 导入完成！数据库迁移成功 🎉")
else:
    print(f"⚠️ 导入返回码: {result.returncode}，部分 SQL 可能被跳过。")


🟡 正在导入到新数据库（改进版：忽略权限错误 + 关闭外键检查）...
STDERR（错误信息片段）:
 mysql: [Warning] Using a password on the command line interface can be insecure.
ERROR 1227 (42000) at line 25: Access denied; you need (at least one of) the SUPER or SYSTEM_VARIABLES_ADMIN privilege(s) for this operation

✅ 导入完成！数据库迁移成功 🎉
